Noisy signal LASSO reconstruction plot as a function of the regularization strength, Gaussian vs row-orthogonal matrices

In [1]:
import numpy as np
from VAMP_toolbox_clean import *
from sklearn import linear_model
from math import isnan

In [ ]:
# Gaussian case, all i.i.d

n_exp = 10
reg_vec = np.linspace(0.01,0.1,n_exp)
E_vec = np.zeros(n_exp)
rho = 0.3
delta0 = 0.01
n,d = 200,100
    
reg = 0.01
avg = 10000

for i in range(n_exp):
    reg = reg_vec[i]
    print(reg,'regularization')
    E_exp = np.zeros(avg)
    for k in range(avg):
        F_train = np.random.normal(0,np.sqrt(1/n),(n,d))
        x0 = gauss_bernoulli(rho,d)
        w = np.random.normal(0,np.sqrt(delta0),n)
        y_train = F_train@x0+w
        clf = linear_model.ElasticNet(alpha=reg/n,l1_ratio=1,max_iter = 100000,tol = 0.00001)
        clf.fit(F_train,y_train)
        x_train = clf.coef_
        E_exp[k] = np.mean((x0-x_train)**2)
    
        #print(E_exp,np.mean(E_exp),np.var(E_exp))
        
    E_vec[i] = np.mean(E_exp)

In [ ]:
# Asymptotics using SE equations/replica prediction

asp = 2
my_eps = 0
niter = 100
damp_se = 1

n_th = 250
reg_vec_th = np.linspace(0.01,0.1,n_th)
E_vec_th = np.zeros(n_th)
    
#reg = reg_vec_th[i]

for k in range(n_th):
    
    print(k)
    
    reg = reg_vec_th[k]
    alpha1_vec,alpha2_vec,eta1_vec,eta2_vec,gamma1_vec,gamma2_vec,E1_vec,E2_vec,tau1_vec,tau2_vec = VAMP_SE_l1_unif(asp,rho,delta0,reg,niter,damp_se,my_eps,0)
    
    V = 1/2*(1/eta1_vec[-1]+1/eta2_vec[-1])
    A1 = alpha1_vec[-1]/V
    A2 = alpha2_vec[-1]/V
    tau1 = tau1_vec[-1]
    tau2 = tau2_vec[-1]
    E = 1/2*(E1_vec[-1]+E2_vec[-1])
    
    while isnan(E) == True:  # restart the iteration in case the fixed point was not found
        print('Restarting iteration')
        alpha1_vec,alpha2_vec,eta1_vec,eta2_vec,gamma1_vec,gamma2_vec,E1_vec,E2_vec,tau1_vec,tau2_vec = VAMP_SE_l1_unif(asp,rho,delta0,reg,niter,damp_se,my_eps,0)
        
        V = 1/2*(1/eta1_vec[-1]+1/eta2_vec[-1])
        A1 = alpha1_vec[-1]/V
        A2 = alpha2_vec[-1]/V
        tau1 = tau1_vec[-1]
        tau2 = tau2_vec[-1]
        E = 1/2*(E1_vec[-1]+E2_vec[-1])
    
    
    
    E_vec_th[k] = E
    print(E)

In [ ]:
# Row-orthogonal setup, experiment

reg_vec = np.linspace(0.01,0.1,n_exp)
E_vec_ort = np.zeros(n_exp)
rho = 0.3
delta0 = 0.01
n,d = 200,100
asp = 2
    
reg = 0.01
avg = 10000

for i in range(n_exp):
    reg = reg_vec[i]
    print(reg,'regularization')
    for k in range(avg):
        #if (k-k//1000*1000)==0:
            #print(k/avg*100,'%')
        F_train,D = build_matrix(1,2,n,d,asp,0)
        x0 = gauss_bernoulli(rho,d)
        w = np.random.normal(0,np.sqrt(delta0),n)
        y_train = F_train@x0+w
        clf = linear_model.ElasticNet(alpha=reg/n,l1_ratio=1,max_iter = 100000,tol = 0.00001)
        clf.fit(F_train,y_train)
        x_train = clf.coef_
        E_exp[k] = np.mean((x0-x_train)**2)
    
        #print(E_exp,np.mean(E_exp),np.var(E_exp))
        
    E_vec_ort[i] = np.mean(E_exp)

In [ ]:
# Row orthogonal theory using the SE asymptotics/replica prediction


asp = 2
my_eps = 0
niter = 100
damp_se = 1
a,b=1,2


reg_vec_th = np.linspace(0.01,0.1,n_th)
E_vec_ort_th = np.zeros(n_th)
    
#reg = reg_vec_th[i]

for k in range(n_th):
    
    print(k)
    reg = reg_vec_th[k]
    alpha1_vec,alpha2_vec,eta1_vec,eta2_vec,gamma1_vec,gamma2_vec,E1_vec,E2_vec,tau1_vec,tau2_vec = VAMP_SE_l1_delta(asp,rho,delta0,reg,niter,damp_se,my_eps)
    
    V = 1/2*(1/eta1_vec[-1]+1/eta2_vec[-1])
    A1 = alpha1_vec[-1]/V
    A2 = alpha2_vec[-1]/V
    tau1 = tau1_vec[-1]
    tau2 = tau2_vec[-1]
    E = 1/2*(E1_vec[-1]+E2_vec[-1])
    
    while isnan(E) == True:
        print('Restarting iteration')
        alpha1_vec,alpha2_vec,eta1_vec,eta2_vec,gamma1_vec,gamma2_vec,E1_vec,E2_vec,tau1_vec,tau2_vec = VAMP_SE_l1_delta(asp,rho,delta0,reg,niter,damp_se,my_eps)
        
        V = 1/2*(1/eta1_vec[-1]+1/eta2_vec[-1])
        A1 = alpha1_vec[-1]/V
        A2 = alpha2_vec[-1]/V
        tau1 = tau1_vec[-1]
        tau2 = tau2_vec[-1]
        E = 1/2*(E1_vec[-1]+E2_vec[-1])
        
    E_vec_ort_th[k] = E    
    print(E)

In [ ]:
plt.plot(reg_vec,E_vec,'+g',label = 'standard Gaussian experiment')
plt.plot(reg_vec_th,E_vec_th,'g', label = 'standard Gaussian theory')
plt.legend()
#plt.savefig('kabashima', dpi=500, quality = 95)
plt.plot(reg_vec,E_vec_ort,'xb',label = 'row orthogonal experiment')
plt.plot(reg_vec_th,E_vec_ort_th,'b', label = 'row orthogonal theory')
plt.legend(fontsize = 12)
#plt.title('Gaussian sanity check d=100')
plt.xlabel('$\lambda_{1}$')
plt.ylabel(r'$\frac{1}{N}||x_{0}-\hat{x}||_{2}^{2}$')
plt.rc('axes', labelsize = 15)
plt.tight_layout()
plt.savefig('kabashima', dpi=500, quality = 95)